In [23]:
from rdkit import Chem
import os
import pandas as pd

folder = "../Data/other/SDF poses"
data = []

for file in os.listdir(folder):
    if file.endswith(".sdf"):
        base_name = file[:-4]  # remove '.sdf'
        suppl = Chem.SDMolSupplier(os.path.join(folder, file))
        for i, mol in enumerate(suppl):
            if mol:
                smiles = Chem.MolToSmiles(mol)
                data.append({"name": base_name, "smiles": smiles})

df = pd.DataFrame(data)

#save to csv
df.to_csv("../Data/docs/7.compounds_for_validaton.csv", index=False)

Index(['smiles'], dtype='object')

In [25]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import os

#first remove old isntances of the sdf file
if os.path.exists("all_conformers.sdf"):
    os.remove("all_conformers.sdf")

df = pd.read_csv("../Data/docs/7.compounds_for_validaton.csv")
writer = Chem.SDWriter("all_conformers.sdf")

for idx, row in df.iterrows():
    mol = Chem.MolFromSmiles(row['smiles'])
    mol = Chem.AddHs(mol)
    mol.SetProp("_Name", str(row['name']))
    ids = AllChem.EmbedMultipleConfs(mol, numConfs=50, randomSeed=42)
    AllChem.UFFOptimizeMoleculeConfs(mol)
    
    for conf_id in ids:
        writer.write(mol, confId=conf_id)

writer.close()

In [ ]:
import pandas as pd
from rdkit import Chem
from dimorphite_dl import run_with_mol

df = pd.read_csv("../Data/docs/6.compounds for vina smiles.csv")
writer = Chem.SDWriter("all_protomers.sdf")

for idx, row in df.iterrows():
    mol = Chem.MolFromSmiles(row['SMILES'])
    protomers = run_with_mol(mol, pH=7.4, min_ph=6.0, max_ph=8.0)

    for protomer in protomers:
        protomer = Chem.AddHs(protomer)
        Chem.EmbedMolecule(protomer)
        writer.write(protomer)

writer.close()